3.	Implement strategies such as oversampling, undersampling, or using weighted loss functions to handle class imbalances. Implement a hyperparameter optimization pipeline for your MLP model using techniques like grid search or random search.
a.	Explore the impact of varying hyperparameters (e.g., learning rate, number of hidden layers, batch size) on model performance.
b.	Compare the training time and performance of your GPU-accelerated implementation with the CPU-based version.

In [7]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score as accuracy, classification_report

dataset = load_breast_cancer()

data = pd.DataFrame(dataset.data, columns = dataset.feature_names)
data['class'] = dataset.target

count_class_1, count_class_0 = data['class'].value_counts()
data_class_0 = data[data['class'] == 0]
data_class_1 = data[data['class'] == 1]

data_class_1_under = data_class_1.sample(count_class_0)
dataset_under = pd.concat([data_class_0, data_class_1_under], axis = 0)
X_under = dataset_under.drop("class", axis = 'columns')
y_under = dataset_under['class']
X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under, y_under, test_size=0.2, stratify=y_under)


data_class_0_over = data_class_0.sample(count_class_1, replace = True)
dataset_over = pd.concat([data_class_0_over, data_class_1], axis = 0)
X_over = dataset_over.drop("class", axis = 'columns')
y_over = dataset_over['class']
X_over_train, X_over_test, y_over_train, y_over_test = train_test_split(X_over, y_over, test_size=0.2, stratify=y_over)


RANDOMIZED SEARCH CV (UNDER SAMPLING)

In [8]:
mlp = MLPClassifier(max_iter=500)

hyperparameters = {
    'hidden_layer_sizes': [(20,), (50, 100), (20, 50, 70)],
    'learning_rate_init': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64]
}

RandomizedSearch = RandomizedSearchCV(mlp, hyperparameters, cv=5, n_jobs=-1)
start_time = time.time()
RandomizedSearch.fit(X_under_train, y_under_train)
end_time = time.time()

best_parameters = RandomizedSearch.best_params_

mlp_cpu = MLPClassifier(**best_parameters, max_iter=100)
start_time_cpu = time.time()
mlp_cpu.fit(X_under_train, y_under_train)
end_time_cpu = time.time()

predictions_cpu = mlp_cpu.predict(X_under_test)
accuracy_cpu = accuracy(y_under_test, predictions_cpu)
print("CPU PERFORMANCE:-")
print("Accuracy:- ", accuracy_cpu)
print(classification_report(y_under_test, predictions_cpu))

print(f"CPU Training Time:- {end_time_cpu - start_time_cpu:.2f}s")

if tf.test.is_gpu_available():
    tf.keras.backend.clear_session()

    X_train = tf.constant(X_under_train, dtype = tf.float32)
    y_train = tf.constant(y_under_train, dtype = tf.float32)
    X_test = tf.constant(X_under_test, dtype = tf.float32)

    mlp_gpu = Sequential([
        Dense(80, activation='relu', input_shape= (X_train.shape[1],)),
        Dense(40, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    mlp_gpu.compile(optimizer=tf.keras.Adam(learning_rate = best_parameters['learning_rate)init']),
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
    
    start_time_gpu = time.time()
    mlp_gpu.fit(X_train, y_train, epochs = 200, batch_size = bet_parametera['batch_size'], verbose=0)
    ens_time_gpu = time.time()

    predictions_gpu = mlp_gpu.predict(X_test)
    predictions_gpu = np.round(prediction_gpu).flatten().astype(int)
    accuracy_gpu = accuracy(y_under_test, predictions_gpu)
    print("GPU PERFORMANCE:- ")
    print("Accuracy:- ", accuracy_gpu)
    print(classification_report(y_under_test, predictions_gpu))
    print(f"Training Time:- {end_time_gpu - start_time_gpu:2.f}s")

else:
    print("GPU not available")


CPU PERFORMANCE:-
Accuracy:-  0.8941176470588236
              precision    recall  f1-score   support

           0       0.84      0.98      0.90        42
           1       0.97      0.81      0.89        43

    accuracy                           0.89        85
   macro avg       0.90      0.90      0.89        85
weighted avg       0.91      0.89      0.89        85

CPU Training Time:- 0.11s
GPU not available


RANDOMIZED SEARCH CV (OVER SAMPLING)

In [9]:
mlp = MLPClassifier(max_iter=500)

hyperparameters = {
    'hidden_layer_sizes': [(20,), (50, 100), (20, 50, 70)],
    'learning_rate_init': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64]
}

RandomizedSearch = RandomizedSearchCV(mlp, hyperparameters, cv=5, n_jobs=-1)
start_time = time.time()
RandomizedSearch.fit(X_over_train, y_over_train)
end_time = time.time()

best_parameters = RandomizedSearch.best_params_

mlp_cpu = MLPClassifier(**best_parameters, max_iter=100)
start_time_cpu = time.time()
mlp_cpu.fit(X_over_train, y_over_train)
end_time_cpu = time.time()

predictions_cpu = mlp_cpu.predict(X_over_test)
accuracy_cpu = accuracy(y_over_test, predictions_cpu)
print("CPU PERFORMANCE:-")
print("Accuracy:- ", accuracy_cpu)
print(classification_report(y_over_test, predictions_cpu))

print(f"CPU Training Time:- {end_time_cpu - start_time_cpu:.2f}s")

if tf.test.is_gpu_available():
    tf.keras.backend.clear_session()

    X_train = tf.constant(X_over_train, dtype = tf.float32)
    y_train = tf.constant(y_over_train, dtype = tf.float32)
    X_test = tf.constant(X_over_test, dtype = tf.float32)

    mlp_gpu = Sequential([
        Dense(80, activation='relu', input_shape= (X_train.shape[1],)),
        Dense(40, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    mlp_gpu.compile(optimizer=tf.keras.Adam(learning_rate = best_parameters['learning_rate)init']),
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
    
    start_time_gpu = time.time()
    mlp_gpu.fit(X_train, y_train, epochs = 200, batch_size = bet_parametera['batch_size'], verbose=0)
    ens_time_gpu = time.time()

    predictions_gpu = mlp_gpu.predict(X_test)
    predictions_gpu = np.round(prediction_gpu).flatten().astype(int)
    accuracy_gpu = accuracy(y_overr_test, predictions_gpu)
    print("GPU PERFORMANCE:- ")
    print("Accuracy:- ", accuracy_gpu)
    print(classification_report(y_over_test, predictions_gpu))
    print(f"Training Time:- {end_time_gpu - start_time_gpu:2.f}s")

else:
    print("GPU not available")


CPU PERFORMANCE:-
Accuracy:-  0.9370629370629371
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        72
           1       0.92      0.96      0.94        71

    accuracy                           0.94       143
   macro avg       0.94      0.94      0.94       143
weighted avg       0.94      0.94      0.94       143

CPU Training Time:- 0.26s
GPU not available
